In [38]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv("analysis_id(in).csv")

In [40]:
df.head()

,Unnamed: 0,date_hour,client,device_type,time_zone,os_name,browser_name,ad_unit,size,bidder,...,sum_time_to_respond,median,min_bid,max_bid,avg_bid,sum_2nd_highest_bid,sum_prebid_winning_bid,sum_winning_bid,sum_nobid,sum_timeout
0,0,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-Footers,728x90,onemobile,...,6614.0,NaN,0.58520,0.58520,0.585200,0.5852,0.5852,NaN,NaN,NaN
1,1,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-Footers,728x90,rhythmone,...,5171.0,NaN,0.33945,0.34875,0.344100,NaN,NaN,NaN,NaN,NaN
2,2,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-InLine-1,300x250,undertone,...,1523.0,NaN,0.96000,0.96000,0.960000,NaN,NaN,NaN,NaN,NaN
3,3,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-O-300x250-1,300x250,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-O-300x250-1,300x250,rhythmone,...,3570.0,0.34875,0.34410,0.34875,0.346425,NaN,NaN,NaN,NaN,NaN


### Analyse the dataset to answer which bidder has the highest win rate (wins / total_bid). 

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              5000 non-null   int64  
 1   date_hour               5000 non-null   object 
 2   client                  5000 non-null   object 
 3   device_type             5000 non-null   object 
 4   time_zone               5000 non-null   object 
 5   os_name                 5000 non-null   object 
 6   browser_name            5000 non-null   object 
 7   ad_unit                 5000 non-null   object 
 8   size                    5000 non-null   object 
 9   bidder                  5000 non-null   object 
 10  bid_range               5000 non-null   object 
 11  media_type              5000 non-null   object 
 12  request_count           5000 non-null   int64  
 13  response_count          5000 non-null   int64  
 14  prebid_win_count        1167 non-null   

In [42]:
df = df[['bidder', 'response_count','prebid_win_count', 'win_count']]

In [43]:
df.head()

,bidder,response_count,prebid_win_count,win_count
0,onemobile,4,1.0,NaN
1,rhythmone,3,NaN,NaN
2,undertone,1,NaN,NaN
3,,0,NaN,NaN
4,rhythmone,2,NaN,NaN


In [44]:
df['bidder'] = df['bidder'].fillna("bidder_x")

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bidder            5000 non-null   object 
 1   response_count    5000 non-null   int64  
 2   prebid_win_count  1167 non-null   float64
 3   win_count         186 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 156.4+ KB


In [46]:
null_values = df.isnull().sum()

In [47]:
null_values

bidder                 0
response_count         0
prebid_win_count    3833
win_count           4814
dtype: int64

In [48]:
df['prebid_win_count'] = df['prebid_win_count'].fillna(0)
df['win_count']  = df['win_count'].fillna(0)

In [49]:
null_values = df.isnull().sum()
null_values

bidder              0
response_count      0
prebid_win_count    0
win_count           0
dtype: int64

In [50]:
df_grp_bidder = df.groupby(['bidder']).agg(
    count_bid = ('response_count', 'sum'),
    count_bid_win = ('prebid_win_count', 'sum')
).reset_index()

In [51]:
df_grp_bidder

,bidder,count_bid,count_bid_win
0,,0,0.0
1,33across,886,136.0
2,appnexus,830,175.0
3,criteo,3181,587.0
4,emx_digital,393,56.0
5,ix,3051,184.0
6,medianet,2556,253.0
7,onemobile,2726,759.0
8,openx,1165,86.0
9,pubmatic,2335,255.0


In [52]:
df_grp_bidder['win_rate'] = df_grp_bidder['count_bid_win']/df_grp_bidder['count_bid']
df_grp_bidder['win_rate'] = df_grp_bidder['win_rate']*100

In [53]:
df_grp_bidder.head()

,bidder,count_bid,count_bid_win,win_rate
0,,0,0.0,NaN
1,33across,886,136.0,15.349887
2,appnexus,830,175.0,21.084337
3,criteo,3181,587.0,18.453317
4,emx_digital,393,56.0,14.249364


In [54]:
df_grp_bidder = df_grp_bidder.dropna()

In [55]:
df_grp_bidder['rank'] = df_grp_bidder['win_rate'].rank(method='dense', ascending=False)

C:\Users\nizam\AppData\Local\Temp\ipykernel_12996\3039173571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp_bidder['rank'] = df_grp_bidder['win_rate'].rank(method='dense', ascending=False)


In [56]:
df_grp_bidder.sort_values('rank',inplace=True)

C:\Users\nizam\AppData\Local\Temp\ipykernel_12996\3198812645.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp_bidder.sort_values('rank',inplace=True)


In [57]:
df_grp_bidder

,bidder,count_bid,count_bid_win,win_rate,rank
17,undertone,104,52.0,50.000000,1.0
15,teads,700,323.0,46.142857,2.0
10,pulsepoint,191,71.0,37.172775,3.0
13,sharethrough,332,104.0,31.325301,4.0
7,onemobile,2726,759.0,27.842993,5.0
11,rhythmone,655,160.0,24.427481,6.0
2,appnexus,830,175.0,21.084337,7.0
3,criteo,3181,587.0,18.453317,8.0
1,33across,886,136.0,15.349887,9.0
4,emx_digital,393,56.0,14.249364,10.0


In [59]:
df_grp_bidder[df_grp_bidder['rank'].isin([1, 2])]

,bidder,count_bid,count_bid_win,win_rate,rank
17,undertone,104,52.0,50.000000,1.0
15,teads,700,323.0,46.142857,2.0


Part 1: Describe how you would build a logistic regression model to predict the probability of a bid winning (win) based on the other columns. What features would you select and why?

Part 2: Based on the following results from a logistic regression model  trained on the dataset, interpret the coefficients:
bid_amount: Coefficient = 0.8
time_to_bid: Coefficient = -0.4
ad_unit_A: Coefficient = 0.3 (reference category: ad_unit_B)

Part 1:
Features selected

1. Datehour - The bid placed in the particular hour may lead to the wining of bid. We can update it to UTC for consistency.
2. bid_amount - The bid amount spent on the bid can be related to ad getting placed. Eg. With increase in bid amount the ad can be placed (assumption).
The different features of bidamount can also be added like - median, mean and bid-range.
3. ad_unit - bidding on placing the ads on footers might give the ad placed, or other permutations like placing on side bar can increase the chance of ad getting placed.  
4. size - bidding on a certain size can secure the ad placed. Like 728*90 is a better size to success of ad placement.
5. response_count - depending upon the response count, if the ads are getting placed. 
6. prebid win count - With wining in prebid, it can be related to the ad placement. So prebid seems to be related to the ad placement
7. browser - if certain browser is favouring certain kind of ad placement.

Part 2:


1. bid_amount, coefficient = 0.8
so with increase in one unit of bidamout, the odds of success i.e p(y=1)/p(y=0), (p(getting add placed)/p(ad not placed)) will get multiplied by e^0.8 (which would be positive number > 0).
Since, 
odds of succes are related to the probabilty of getting the success, therefore the the probability of getting the success will increase.

2.  time_to_bid, coefficient = -0.4
so with the increase in one unit of time to bid, the odd of success, will get multiplied by e^-0.4, and as the coefficient if a negative number therfore the odds of success would be reduced with increase in time_to_bid.
As odds of success of getting ad placed get reduced therefore the probablity of getting the ad placed also reduces.


3. ad_unit_A, coefficient = 0.3
So with increase in one unit of ad_unit, the odds of success will get multiplied by e^03, as 0.3 > 0.8(comparing from first example), 
And it will increase the odds of success but less compared to the coefficient 0.8, 
and with the increase in odds of success the probability of getting the success will also increase.

